# Traffic incident detection with real time traffic data 

Incidents on highways can significantly affect the road capacity and decrease the reliability of the highway system. This project proposes a tool for incident detection, which solely utilizes data from traffic counters and could therefore provide an alternative solution in regions where sensors and cameras are limited. The occurrence of incidents on the network typically causes delay and reduction in traffic volume, which is recorded by traffic counters and reflected in real-time traffic data. In this research, we develop an algorithm that uses Machine Learning Gaussian Process regression to extract the average traffic flow parameters and identifies anomalous data with a statistical approach. 

In order to test the precision and recall of the algorithm, we examined the correlation between anomalous data and incident records and built a proof-of-concept model with traffic data from a segment of the M1 motorway. 


There are five main parts of the this notebook
* Data processing
* Model trainning
* Anomoly detection
* Anomoly filtering
* Precision and Recall Calculation



## Section 1 import traffic counter data and incident detection data

### Import traffic counter data

In [1]:
import pandas as pd    
import numpy as np
from matplotlib import pyplot as plt

In [2]:
listOfFile=["190115.tcd.csv","190122.tcd.csv","190129.tcd.csv","190205.tcd.csv","190212.tcd.csv","190219.tcd.csv",
"190226.tcd.csv","190305.tcd.csv","190312.tcd.csv","190319.tcd.csv","190402.tcd.csv","190409.tcd.csv","190416.tcd.csv",
"190423.tcd.csv","190430.tcd.csv","190507.tcd.csv","190514.tcd.csv","190521.tcd.csv","190528.tcd.csv","190604.tcd.csv",
"190611.tcd.csv","190618.tcd.csv","190625.tcd.csv","190702.tcd.csv","190709.tcd.csv","190716.tcd.csv","190723.tcd.csv",
"190730.tcd.csv","190813.tcd.csv","190820.tcd.csv","190827.tcd.csv","190903.tcd.csv","190910.tcd.csv","190917.tcd.csv",
"190924.tcd.csv","191001.tcd.csv","191008.tcd.csv","191022.tcd.csv","191029.tcd.csv","191105.tcd.csv","191112.tcd.csv",
"191119.tcd.csv","191203.tcd.csv","191210.tcd.csv","191217.tcd.csv"]

In [3]:
sum_flow_yearly=None
for i in range(len(listOfFile)):
    df = pd.read_csv(listOfFile[i])
    #for traffic counter M1/2393A
    df_2393A=df.iloc[0:1440]
    a=df_2393A.loc[:,'Flow(Lane 1)'].to_numpy()
    b=df_2393A.loc[:,'Flow(Lane 2)'].to_numpy()
    c=df_2393A.loc[:,'Flow(Lane 3)'].to_numpy()
    d=df_2393A.loc[:,'Flow(Lane 4)'].to_numpy()
    sum_flow_daily=a+b+c+d
    if sum_flow_yearly is None:
        sum_flow_yearly=sum_flow_daily
    else:
        sum_flow_yearly=np.vstack((sum_flow_yearly,sum_flow_daily))  

### Import incident records

In [4]:
inc_data = pd.read_excel ('incident records.xlsx' )

In [5]:
inc_data = pd.DataFrame(inc_data)
inc_data_tue = inc_data[(inc_data['Day_of_week'] == 'Tuesday') & (inc_data['Northing'] < 219789)& (inc_data['Northing'] > 205973)]
inc_data_sim = inc_data_tue.loc[:,'Minute_of_hour':'Month_of_year']

In [6]:
increc_hour=inc_data_sim.loc[:,'Hour_of_day'].to_numpy()
increc_day=inc_data_sim.loc[:,'Day_of_month'].to_numpy()
increc_month=inc_data_sim.loc[:,'Month_of_year'].to_numpy()
increc_mins=inc_data_sim.loc[:,'Minute_of_hour'].to_numpy()
datasum=[]
for i in range(len(increc_hour)):
    if len(str(increc_day[i]))==1:
        increc_day_str='0'+str(increc_day[i])
    else:
        increc_day_str=str(increc_day[i])
    data=str(increc_month[i])+increc_day_str
    datasum.append(data)
increctime=increc_hour+increc_mins/60
# convert the data into a comparable format
increc_date=np.transpose([datasum,increctime])

## Section 2 Define functions

### Scatter plot of traffic counter

In [7]:
def datacondensing(orginal,compactingfactor):
    a= orginal.shape[1]
    mergedsize=int(a/compactingfactor)
    newdataset=None
    if a%compactingfactor==0:
        for i in range (mergedsize):
            mergeddata=np.sum(orginal[:,compactingfactor*(i):compactingfactor*(i+1)], axis=1)
            if newdataset is None:
                newdataset=mergeddata
            else:
                newdataset=np.vstack((newdataset,mergeddata))
        return (newdataset.transpose())
    else:
        print ('ERROR! aliquant, please choose a factor of 1440')

In [8]:
def scatterplot(dataset,compactingfactor):
    dataset=datacondensing(dataset,compactingfactor)
    for i in range (len(dataset)):
        x=np.linspace(0,int(1440/(compactingfactor)),int(1440/(compactingfactor)))/(60/compactingfactor)
        dailyflow=dataset[i,:]
        plt.plot(x,dailyflow,'.')
    plt.xlabel('Hours in a day', fontsize=18)
    plt.ylabel('Traffic volumn per '+str(compactingfactor)+' min ', fontsize=16)
    plt.xlim(0, 24) 
    plt.show()

In [9]:
def zero_to_nan(values):
    """Replace every 0 with 'nan' and return a copy."""
    return [float('nan') if x==0 else x for x in values]

In [10]:
def simpleGDforanomolies(dataset,compactingfactor):
    dataset=datacondensing(dataset,compactingfactor)
    Mu=np.mean(dataset,axis=0)
    var=np.var(dataset,axis=0)
    lower_limite=Mu-2*np.sqrt(var)
    upper_limite=Mu+2*np.sqrt(var)
    anom_year=None
    x=np.linspace(0,int(1440/(compactingfactor)),int(1440/(compactingfactor)))/(60/compactingfactor)
    for i in range (len(dataset)):
        dailyflow=dataset[i,:]
        anom_day=(dailyflow<lower_limite).astype(int)
        if anom_year is None:
            anom_year=anom_day
        else:
            anom_year=np.vstack((anom_year,anom_day))
    for i in range (len(dataset)):
        dailyflow=dataset[i,:]
        #plt.plot(dailyflow,'.')
        anomonly=dailyflow*anom_year[i,:]
        plt.plot(x,zero_to_nan(anomonly),'.')
    plt.plot(x,lower_limite,color='black')
    plt.plot(x,upper_limite,color='black')
    plt.plot(x,Mu,color='black')
    plt.xlabel('Hours in a day', fontsize=18)
    plt.xlim(0, 24) 
    plt.ylabel('Traffic volumn per '+str(compactingfactor)+' min ', fontsize=16)
    plt.show()

In [11]:
def anom_filter (value,threshold):
    # imputs are boolean of function and threshold value
    tot=0
    threholded=np.zeros(value.shape)
    for i in range(len(value)):
        if value[i]==0:
            tot=tot*value[i]
        else:
            tot=tot+value[i]
        if tot==threshold:
            threholded[i]=1
            tot=0
    return threholded
    anom_year=None

In [12]:
def getthelistofanomalies(value):
    list_det_= None
    time_list=np.asarray(np.nonzero(value))
    #print ("total number of anomolies detected: ",time_list.shape[1])
    for i in range (time_list.shape[1]):
        date_detected=listOfFile[time_list[0,i]].split('.')[0]
        day=date_detected[2]+date_detected[3]+date_detected[4]+date_detected[5]
        time=time_list[1,i]/60;
        inc_detected=[int(day),time]
        if list_det_ is None:
            list_det_=inc_detected
        else:
            list_det_=np.vstack((list_det_,inc_detected))
    return list_det_

In [13]:
def precision (detected, records):
    
    FP=0
    error=0.5
    for i in range(len(detected)):
        datefinding=np.where(records[:,0] == str(int(detected[i,0])))
        datefinding=np.asarray(datefinding)
        #print(type(datefinding))
        #print(datefinding)
        is_empty = datefinding.size == 0
        if is_empty:
            #print("incident record", i, "nothing matches")
            FP=FP+1
        else:
            timeinfounddata=np.array([])
            for j in range(records[datefinding,1].shape[1]):
                whattime = float(records[datefinding,1][0,j])
                timeinfounddata=np.append(timeinfounddata,whattime)
            timefinding=np.where(np.logical_and(timeinfounddata>=detected[i,1]-error,timeinfounddata<=detected[i,1]+error))
            timefinding=np.asarray(timefinding)
            is_time_empty = timefinding.size == 0
            if is_time_empty:
                #print("incident record", i,"nothing matches")
                FP=FP+1
            #else:
                #print("incident record", i,detected[i,1],"found matched value", "time is:",timeinfounddata[timefinding]  )
    
    precision=1-FP/len(detected)
    
    return precision
    #print ("precision rate is:", 1-FP/len(detected))

In [14]:
def plotfilterreddata(dataset,threshold,compactingfactor,records):
    value=datacondensing(dataset,compactingfactor)
    Mu=np.mean(value,axis=0)
    var=np.var(value,axis=0)
    lower_limite=Mu-2*np.sqrt(var)
    upper_limite=Mu+2*np.sqrt(var)
    anom_year = None
    x=np.linspace(0,int(1440/(compactingfactor)),int(1440/(compactingfactor)))/(60/compactingfactor)
    for i in range (len(value)):
        dailyflow=value[i,:]
        anom_day= anom_filter((dailyflow<lower_limite).astype(int),threshold)
        if anom_year is None:
            anom_year=anom_day
        else:
            anom_year=np.vstack((anom_year,anom_day))
    for i in range (len(value)):
        dailyflow=value[i,:]
        anomonly=dailyflow*anom_year[i,:]
        '''
        plt.plot(x,zero_to_nan(anomonly),'.')
    plt.plot(x,lower_limite,color='black')
    plt.plot(x,upper_limite,color='black')
    plt.plot(x,Mu,color='black')
    plt.xlim(0, 24)
    plt.xlabel('mins in a day', fontsize=18)
    plt.ylabel('Traffic volumn per '+str(compactingfactor)+' min ', fontsize=16)
    plt.show()
    '''
    detectedanomalies=getthelistofanomalies(anom_year)
    pre=precision (detectedanomalies, records)
    return(pre)



In [15]:
plotfilterreddata(sum_flow_yearly,4,5,increc_date)

0.07999999999999996

In [16]:
X=[1,2,3,4,5,6,10,15]
Y=[1,2,3,4,5]

In [17]:
Z=np.zeros((len(X), len(Y)))
for i in range (len(X)):
    for j in range (len(Y)):
        Z[i,j]=plotfilterreddata(sum_flow_yearly,Y[j],X[i],increc_date)

In [18]:
np.amax(Z)

0.6486486486486487